In [ ]:
#1
import numpy as np
import matplotlib.pyplot as plt

# Kalman Filter implementation
class KalmanFilter:
    def __init__(self, initial_state, initial_uncertainty, measurement_uncertainty, process_uncertainty):
        # Initial state (position and velocity)
        self.state = initial_state  # [position, velocity]
        
        # Initial state uncertainty (covariance matrix)
        self.P = initial_uncertainty
        
        # Measurement uncertainty
        self.R = measurement_uncertainty
        
        # Process uncertainty (how much we trust our model)
        self.Q = process_uncertainty
        
        # State transition matrix (assuming constant velocity model)
        self.F = np.array([[1, 1],  # Position update: x(t) = x(t-1) + v(t-1)
                           [0, 1]]) # Velocity update: v(t) = v(t-1)
        
        # Measurement matrix (we directly measure position)
        self.H = np.array([[1, 0]])  # We only measure position, not velocity
    
    def predict(self):
        # Prediction Step
        self.state = np.dot(self.F, self.state)  # Predicted state
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q  # Predicted uncertainty

    def update(self, measurement):
        # Update Step
        # Compute the Kalman gain
        S = np.dot(np.dot(self.H, self.P), self.H.T) + self.R
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))
        
        # Update the state estimate with the measurement
        y = measurement - np.dot(self.H, self.state)  # Measurement residual
        self.state = self.state + np.dot(K, y)
        
        # Update the uncertainty
        self.P = self.P - np.dot(np.dot(K, self.H), self.P)

# Simulation parameters
num_steps = 50  # Number of time steps
true_position = 0  # True initial position
true_velocity = 1  # True initial velocity (constant velocity)

# Generate synthetic noisy measurements
np.random.seed(0)  # For reproducibility
measurements = []
for i in range(num_steps):
    noise = np.random.normal(0, 2)  # Measurement noise
    true_position += true_velocity  # Update true position
    noisy_measurement = true_position + noise
    measurements.append(noisy_measurement)

# Initial state estimate (position, velocity) and uncertainty
initial_state = np.array([0, 1])  # Initial guess for position and velocity
initial_uncertainty = np.array([[1000, 0],  # High uncertainty about initial state
                                [0, 1000]])

# Measurement uncertainty and process uncertainty
measurement_uncertainty = 4  # The variance in our measurements (e.g., sensor noise)
process_uncertainty = np.array([[1, 0],  # Assume some process noise for both position and velocity
                                [0, 1]])

# Create the Kalman Filter object
kf = KalmanFilter(initial_state, initial_uncertainty, measurement_uncertainty, process_uncertainty)

# Apply Kalman filter to measurements
estimated_positions = []
estimated_velocities = []

for measurement in measurements:
    kf.predict()  # Prediction step
    kf.update(measurement)  # Update step
    estimated_positions.append(kf.state[0])
    estimated_velocities.append(kf.state[1])

# Plot the results
plt.figure(figsize=(12, 6))

# Plot true position, noisy measurements, and estimated position
plt.subplot(1, 2, 1)
plt.plot(estimated_positions, label='Estimated Position', color='b')
plt.scatter(range(num_steps), measurements, color='r', label='Noisy Measurements', alpha=0.5)
plt.plot(range(num_steps), np.arange(num_steps) + true_velocity * np.arange(num_steps), label='True Position', color='g')
plt.xlabel('Time Step')
plt.ylabel('Position')
plt.legend()
plt.title('Position Estimation')

# Plot estimated velocity
plt.subplot(1, 2, 2)
plt.plot(estimated_velocities, label='Estimated Velocity', color='b')
plt.axhline(true_velocity, color='g', linestyle='--', label='True Velocity')
plt.xlabel('Time Step')
plt.ylabel('Velocity')
plt.legend()
plt.title('Velocity Estimation')

plt.tight_layout()
plt.show()


In [ ]:
#2
import numpy as np

def normalize_image(image):
    """
    Normalize the pixel values of an image to be between 0 and 1.
    
    Parameters:
    - image: numpy array of shape (H, W, C) or (H, W), where H is the height,
             W is the width, and C is the number of color channels (3 for RGB).
    
    Returns:
    - Normalized image: numpy array with pixel values between 0 and 1.
    """
    # Ensure the image is in float32 format for accurate division
    normalized_image = image.astype(np.float32)
    
    # Normalize the pixel values to [0, 1]
    normalized_image /= 255.0
    
    return normalized_image


In [ ]:
#3
import numpy as np

def generate_dummy_detections(num_detections, image_shape=(640, 480), confidence_threshold=0.5):
    """
    Generate dummy object detection data with confidence scores and bounding boxes, 
    and filter detections based on a confidence threshold.
    
    Parameters:
    - num_detections: The number of dummy detections to generate.
    - image_shape: Tuple representing the shape of the image (height, width).
    - confidence_threshold: Confidence score threshold for filtering detections.
    
    Returns:
    - filtered_boxes: List of bounding boxes for detections with confidence > threshold.
    - filtered_scores: List of confidence scores for detections with confidence > threshold.
    """
    # Generate random bounding boxes within the image dimensions
    height, width = image_shape
    boxes = np.random.randint(0, min(height, width), size=(num_detections, 4))
    
    # Ensure boxes are valid: (x1 < x2) and (y1 < y2)
    boxes[:, 2] = np.clip(boxes[:, 2], boxes[:, 0] + 1, width)  # x2 should be > x1
    boxes[:, 3] = np.clip(boxes[:, 3], boxes[:, 1] + 1, height)  # y2 should be > y1
    
    # Generate random confidence scores between 0 and 1
    confidence_scores = np.random.rand(num_detections)
    
    # Filter detections based on the confidence threshold
    filtered_indices = confidence_scores > confidence_threshold
    filtered_boxes = boxes[filtered_indices]
    filtered_scores = confidence_scores[filtered_indices]
    
    return filtered_boxes, filtered_scores

# Example usage
num_detections = 10
confidence_threshold = 0.5

# Generate dummy data
boxes, scores = generate_dummy_detections(num_detections, confidence_threshold=confidence_threshold)

# Print the results
print("Bounding Boxes (x1, y1, x2, y2):")
print(boxes)
print("\nConfidence Scores:")
print(scores)


In [ ]:
#4
import numpy as np

def extract_features_from_yolo_detections(detections, feature_dim=128):
    """
    Given a list of YOLO detections, generate a random 128-dimensional feature vector for each detection.
    
    Parameters:
    - detections: List of YOLO detections. Each detection is expected to be a tuple/list
                  in the form (x_center, y_center, width, height, class_id, confidence).
    - feature_dim: The dimensionality of the feature vector (default is 128).
    
    Returns:
    - feature_vectors: A list of 128-dimensional feature vectors, one for each detection.
    """
    feature_vectors = []
    
    # Iterate over each detection and generate a random feature vector
    for detection in detections:
        # Optionally, you could use the detection's properties (e.g., bounding box, class) to influence the feature vector
        # For simplicity, we generate a random feature vector here.
        feature_vector = np.random.rand(feature_dim)
        
        # Append the feature vector to the list
        feature_vectors.append(feature_vector)
    
    return feature_vectors

# Example usage:
# Let's assume we have a list of detections in the form (x_center, y_center, width, height, class_id, confidence)
yolo_detections = [
    (100, 150, 50, 60, 0, 0.98),  # Example detection 1
    (200, 250, 80, 100, 1, 0.88),  # Example detection 2
    (300, 350, 120, 150, 2, 0.93), # Example detection 3
]

# Extract random feature vectors for each detection
feature_vectors = extract_features_from_yolo_detections(yolo_detections)

# Print the generated feature vectors
for i, feature_vector in enumerate(feature_vectors):
    print(f"Detection {i+1} Feature Vector (128 dimensions):\n{feature_vector}\n")


In [ ]:
#5
import numpy as np
from scipy.spatial.distance import cdist

def reidentify_objects(current_features, previous_features, threshold=0.5):
    """
    Re-identify objects by matching feature vectors using Euclidean distance.

    Parameters:
    - current_features: A list/array of current detections' feature vectors (N x D),
                         where N is the number of current detections, and D is the feature vector dimensionality.
    - previous_features: A list/array of previous detections' feature vectors (M x D),
                          where M is the number of previous detections, and D is the feature vector dimensionality.
    - threshold: A distance threshold to consider if two objects are the same. If the distance is below this threshold,
                 they are considered the same object.

    Returns:
    - matched_ids: A list of indices of the matched previous detections for each current detection.
                   If no match is found (distance > threshold), -1 is returned for that detection.
    """
    # Compute the Euclidean distance matrix between current and previous feature vectors
    distances = cdist(current_features, previous_features, metric='euclidean')
    
    # Find the indices of the closest previous detections for each current detection
    matched_ids = []
    
    for i, current_feat in enumerate(distances):
        # Find the index of the minimum distance for the current detection
        min_distance_idx = np.argmin(current_feat)
        min_distance = current_feat[min_distance_idx]
        
        # Check if the minimum distance is below the threshold
        if min_distance < threshold:
            matched_ids.append(min_distance_idx)
        else:
            matched_ids.append(-1)  # No match if distance exceeds the threshold
    
    return matched_ids

# Example usage:
# Let's assume we have a list of feature vectors for current and previous detections
current_features = np.random.rand(5, 128)  # 5 current detections with 128-dimensional feature vectors
previous_features = np.random.rand(8, 128)  # 8 previous detections with 128-dimensional feature vectors

# Re-identify objects based on feature vectors
threshold = 0.5  # Distance threshold for matching objects
matched_ids = reidentify_objects(current_features, previous_features, threshold)

# Print the results
print("Matched IDs (or -1 for no match):")
print(matched_ids)


In [ ]:
#6
import numpy as np
import cv2

class KalmanTracker:
    def __init__(self, initial_state=None):
        # Initialize Kalman Filter
        self.kf = cv2.KalmanFilter(4, 2)  # 4 state variables (x, y, vx, vy), 2 measurements (x, y)
        
        # State transition matrix (Assume constant velocity model)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0],  # x = x + vx
                                             [0, 1, 0, 1],  # y = y + vy
                                             [0, 0, 1, 0],  # vx = vx
                                             [0, 0, 0, 1]], # vy = vy
        
        # Measurement matrix (we only measure position)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0],
                                               [0, 1, 0, 0]], dtype=np.float32)
        
        # Process noise covariance (assume noise)
        self.kf.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
        
        # Measurement noise covariance
        self.kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.1
        
        # Initial state estimate: [x, y, vx, vy]
        if initial_state is not None:
            self.kf.statePost = np.array(initial_state, dtype=np.float32)
        else:
            self.kf.statePost = np.zeros(4, dtype=np.float32)  # Assuming initial position (0, 0) and velocity (0, 0)
    
    def predict(self):
        # Predict the next state based on the current state
        predicted_state = self.kf.predict()
        return predicted_state[:2]  # Return predicted position (x, y)
    
    def update(self, measurement):
        # Update the Kalman filter with a new measurement (detected object position)
        self.kf.correct(measurement)
        return self.kf.statePost[:2]  # Return updated position (x, y)


def track_objects_with_kalman(yolo_detections, initial_positions=None):
    """
    Tracks objects using Kalman Filter based on YOLO detections.
    
    Parameters:
    - yolo_detections: List of detections with bounding boxes in the form (x_center, y_center, width, height)
    - initial_positions: Initial positions of the objects to track (Optional)
    
    Returns:
    - tracked_positions: List of positions for each object after tracking with Kalman Filter
    """
    trackers = []
    tracked_positions = []
    
    # Initialize Kalman trackers for each object
    for i, detection in enumerate(yolo_detections):
        x_center, y_center, _, _ = detection
        
        # If no initial positions are provided, initialize with YOLO center
        if initial_positions is None:
            initial_state = [x_center, y_center, 0, 0]  # Assume no velocity initially
        else:
            initial_state = initial_positions[i]
        
        kalman_tracker = KalmanTracker(initial_state)
        trackers.append(kalman_tracker)
    
    # Update each tracker with the new YOLO detections
    for i, detection in enumerate(yolo_detections):
        x_center, y_center, _, _ = detection
        
        # Create a measurement (we only measure position)
        measurement = np.array([[x_center], [y_center]], dtype=np.float32)
        
        # Update the Kalman filter and get the predicted position
        tracked_position = trackers[i].update(measurement)
        tracked_positions.append(tracked_position)
    
    return tracked_positions


# Example usage:
yolo_detections = [
    (100, 150, 50, 60),  # Example detection (x_center, y_center, width, height)
    (200, 250, 80, 100),
    (300, 350, 120, 150),
]

# Track objects using Kalman Filter
tracked_positions = track_objects_with_kalman(yolo_detections)

# Print the tracked positions
for i, position in enumerate(tracked_positions):
    print(f"Tracked position for object {i+1}: {position}")


In [ ]:
#7
import numpy as np
import matplotlib.pyplot as plt

class KalmanFilter:
    def __init__(self, initial_state, process_noise_cov, measurement_noise_cov):
        # Kalman filter initialization
        self.state = np.array(initial_state, dtype=np.float32)  # [x, y, vx, vy]
        self.process_noise_cov = process_noise_cov  # Process noise covariance
        self.measurement_noise_cov = measurement_noise_cov  # Measurement noise covariance
        
        # Kalman Filter Matrices
        self.transition_matrix = np.array([[1, 0, 1, 0],
                                           [0, 1, 0, 1],
                                           [0, 0, 1, 0],
                                           [0, 0, 0, 1]], dtype=np.float32)
        
        self.measurement_matrix = np.array([[1, 0, 0, 0],
                                            [0, 1, 0, 0]], dtype=np.float32)
        
        self.process_noise_cov_matrix = np.eye(4, dtype=np.float32) * self.process_noise_cov
        self.measurement_noise_cov_matrix = np.eye(2, dtype=np.float32) * self.measurement_noise_cov
        self.state_covariance = np.eye(4, dtype=np.float32)  # Initial state covariance matrix

    def predict(self):
        """ Prediction step of the Kalman filter """
        self.state = np.dot(self.transition_matrix, self.state)  # Predict the next state
        self.state_covariance = np.dot(np.dot(self.transition_matrix, self.state_covariance), self.transition_matrix.T) + self.process_noise_cov_matrix
        return self.state[:2]  # Return predicted position (x, y)

    def update(self, measurement):
        """ Update step of the Kalman filter """
        innovation = measurement - np.dot(self.measurement_matrix, self.state)  # Measurement residual
        innovation_covariance = np.dot(np.dot(self.measurement_matrix, self.state_covariance), self.measurement_matrix.T) + self.measurement_noise_cov_matrix
        
        kalman_gain = np.dot(np.dot(self.state_covariance, self.measurement_matrix.T), np.linalg.inv(innovation_covariance))
        
        self.state = self.state + np.dot(kalman_gain, innovation)  # Correct the predicted state
        self.state_covariance = self.state_covariance - np.dot(np.dot(kalman_gain, self.measurement_matrix), self.state_covariance)  # Update covariance matrix
        
        return self.state[:2]  # Return updated position (x, y)

def simulate_object_movement(true_position, velocity, num_steps, noise_level):
    """ Simulate the movement of an object in 2D space with random noise """
    positions = [true_position]
    for _ in range(num_steps):
        true_position = true_position + velocity  # Update true position
        noisy_position = true_position + np.random.normal(0, noise_level, 2)  # Add noise to the position
        positions.append(noisy_position)
    return np.array(positions)

# Set initial conditions
initial_position = np.array([0, 0], dtype=np.float32)  # Initial position (x, y)
velocity = np.array([1, 1], dtype=np.float32)  # Velocity in (vx, vy)
num_steps = 50  # Number of time steps to simulate
noise_level = 0.5  # Standard deviation of noise
process_noise_cov = 0.01  # Process noise covariance (reflects uncertainty in motion model)
measurement_noise_cov = 1  # Measurement noise covariance (reflects sensor noise)

# Simulate object movement with noise
true_positions = simulate_object_movement(initial_position, velocity, num_steps, noise_level)

# Initialize Kalman Filter
kf = KalmanFilter(initial_state=[0, 0, 1, 1],  # Initial state [x, y, vx, vy]
                  process_noise_cov=process_noise_cov,
                  measurement_noise_cov=measurement_noise_cov)

# Track object using Kalman Filter
predicted_positions = []
updated_positions = []

for noisy_position in true_positions:
    # Predict the next position
    predicted_position = kf.predict()
    predicted_positions.append(predicted_position)
    
    # Update Kalman Filter with noisy measurement
    updated_position = kf.update(noisy_position)
    updated_positions.append(updated_position)

# Convert lists to numpy arrays for easy plotting
predicted_positions = np.array(predicted_positions)
updated_positions = np.array(updated_positions)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(true_positions[:, 0], true_positions[:, 1], label='True Position', color='g', linestyle='-', marker='o', markersize=5)
plt.plot(true_positions[:, 0], true_positions[:, 1], label='Noisy Measurement', color='r', linestyle='--', marker='x', markersize=5)
plt.plot(predicted_positions[:, 0], predicted_positions[:, 1], label='Predicted Position (Kalman)', color='b', linestyle='-.', marker='^', markersize=5)
plt.plot(updated_positions[:, 0], updated_positions[:, 1], label='Updated Position (Kalman)', color='purple', linestyle='-', marker='s', markersize=5)

plt.title("Object Tracking with Kalman Filter")
plt.xlabel("X Position")
plt.ylabel("Y Position")
plt.legend()
plt.grid(True)
plt.show()
